In [12]:
!python --version

Python 3.12.7


In [13]:
!pip install scikit-learn==1.6.1

  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 5.1 MB/s eta 0:00:00a 0:00:01
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 5.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 7.1 MB/s eta 0:00:00a 0:00:01
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


hash = sha256:b4fc2525eca2c69a59260f583c56a7557c6ccdf8deafdba6e060f94c1c59738e

In [9]:
import pickle

In [14]:
model_file = 'pipeline_v1.bin'

with open(model_file, 'rb') as f:
    dv, model = pickle.load(f)

In [16]:
dv

DictVectorizer()

In [17]:
model

LogisticRegression(solver='liblinear')

In [18]:
with open(model_file, 'rb') as f:
    pipeline = pickle.load(f)

In [20]:
pipeline

Pipeline(steps=[('dictvectorizer', DictVectorizer()),
                ('logisticregression', LogisticRegression(solver='liblinear'))])

In [19]:
record = {
        "lead_source": "paid_ads",
        "number_of_courses_viewed": 2,
        "annual_income": 79276.0
    }

# Make prediction
probability = pipeline.predict_proba([record])[0][1]

In [21]:
print(f"Record: {record}")
print(f"Probability of conversion: {probability:.3f}")

Record: {'lead_source': 'paid_ads', 'number_of_courses_viewed': 2, 'annual_income': 79276.0}
Probability of conversion: 0.534


In [22]:
!pip install fastapi uvicorn

  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 6.2 MB/s eta 0:00:00a 0:00:01
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [23]:
# Create FastAPI application code
fastapi_code = '''
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import uvicorn

# Load the pipeline
with open("pipeline_v1.bin", "rb") as f:
    pipeline = pickle.load(f)

# Create FastAPI app
app = FastAPI()

class Lead(BaseModel):
    lead_source: str
    number_of_courses_viewed: int
    annual_income: float

@app.post("/predict")
def predict(lead: Lead):
    # Convert to dictionary
    lead_dict = lead.dict()

    # Make prediction
    probability = pipeline.predict_proba([lead_dict])[0][1]

    return {"probability": probability}

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''

# Save the FastAPI code to a file
with open("app.py", "w") as f:
    f.write(fastapi_code)

print("FastAPI application code saved to app.py")
print("To run the service, use: uvicorn app:app --host 0.0.0.0 --port 8000")

FastAPI application code saved to app.py
To run the service, use: uvicorn app:app --host 0.0.0.0 --port 8000


In [24]:
# Test the prediction locally (without FastAPI server)
# This simulates what the FastAPI service would do

client_data = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}

probability = pipeline.predict_proba([client_data])[0][1]

print(f"Client data: {client_data}")
print(f"Probability of conversion: {probability:.3f}")

Client data: {'lead_source': 'organic_search', 'number_of_courses_viewed': 4, 'annual_income': 80304.0}
Probability of conversion: 0.534


- Install FastAPI: pip install fastapi uvicorn
- Run the service: uvicorn app:app --host 0.0.0.0 --port 8000
- Test with requests:

In [25]:
import requests
url = "http://localhost:8000/predict"
client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}
response = requests.post(url, json=client)
print(response.json())

{'probability': 0.5340417283801275}
